Copyright (c) 2014-2019 National Technology and Engineering
Solutions of Sandia, LLC. Under the terms of Contract DE-NA0003525
with National Technology and Engineering Solutions of Sandia, LLC,
the U.S. Government retains certain rights in this software.

Redistribution and use in source and binary forms, with or without
modification, are permitted provided that the following conditions
are met:

1. Redistributions of source code must retain the above copyright
   notice, this list of conditions and the following disclaimer.

2. Redistributions in binary form must reproduce the above copyright
   notice, this list of conditions and the following disclaimer in the
   documentation and/or other materials provided with the distribution.
  
THIS SOFTWARE IS PROVIDED BY THE COPYRIGHT HOLDERS AND CONTRIBUTORS
"AS IS" AND ANY EXPRESS OR IMPLIED WARRANTIES, INCLUDING, BUT NOT
LIMITED TO, THE IMPLIED WARRANTIES OF MERCHANTABILITY AND FITNESS FOR
A PARTICULAR PURPOSE ARE DISCLAIMED. IN NO EVENT SHALL THE COPYRIGHT
HOLDER OR CONTRIBUTORS BE LIABLE FOR ANY DIRECT, INDIRECT, INCIDENTAL,
SPECIAL, EXEMPLARY, OR CONSEQUENTIAL DAMAGES (INCLUDING, BUT NOT
LIMITED TO, PROCUREMENT OF SUBSTITUTE GOODS OR SERVICES; LOSS OF USE,
DATA, OR PROFITS; OR BUSINESS INTERRUPTION) HOWEVER CAUSED AND ON ANY
THEORY OF LIABILITY, WHETHER IN CONTRACT, STRICT LIABILITY, OR TORT
(INCLUDING NEGLIGENCE OR OTHERWISE) ARISING IN ANY WAY OUT OF THE USE
OF THIS SOFTWARE, EVEN IF ADVISED OF THE POSSIBILITY OF SUCH DAMAGE.


# Trajectory Clustering Example

This notebook is an end-to-end example of how to cluster trajectories in Tracktable using distance geometry.  It goes through the following steps:

1.  Read in points from a file.
2.  Assemble those points into trajectories.
3.  Create a distance geometry signature for each trajectory.
4.  Using those signatures as feature vectors, compute clusters using DBSCAN.
5.  Print statistics about each cluster.
6.  Render the resulting clusters onto a map.

Eventually, distance geometry computation will move into the library itself.

In [1]:
# Set up Matplotlib to render in a notebook before anyone else can change its back end.
import matplotlib
%matplotlib inline

Compute an N-point distance geometry signature    
    
Distance geometry is a technique for characterizing a curve in space by measuring the distances between evenly spaced points (called control points) on the curve. This implementation has three parameters:    
&nbsp;&nbsp;1. A trajectory    
&nbsp;&nbsp;2. The number of control points     
&nbsp;&nbsp;3. Whether to normalize the distances in the signature so that the largest distance is always 1.    
    
The number of control points controls the fidelity of the resulting signature. The more control points, the more accurately the features of the curve can be represented, but the longer it takes to compute.    
    
Normalizing the distance allows shape-based comparison between trajectories by taking the dot product of their respective distance geometry signatures. The higher the dot product, the more similar the trajectories. There are many possible normalization schemes; this is the one we find useful.    
    
Returns:    
&nbsp;&nbsp;tracktable.domain.feature_vectors.FeatureVectorNN where NN is the size of the resulting distance geometry signature. 

In [2]:
from tracktable.domain.feature_vectors import convert_to_feature_vector
from tracktable.core.geomath import point_at_length_fraction
from tracktable.core.geomath import distance

def distance_geometry_signature(trajectory, num_control_points=4, normalize_distance = True):
    # Sets the distance increment for control points based on the number of control points
    # Calculates the fractions of the trajectory where control points should be
    # Gives the values where the control points are located
    control_point_increment = 1.0/(num_control_points-1)    
    control_point_fractions = [control_point_increment * i for i in range(num_control_points)]    
    control_points = [point_at_length_fraction(trajectory, t) for t in control_point_fractions]
    
    # A signature is a collection of the calculated distances that will be converted to a feature vector
    signature = []
    # Calculate the list of distances
    for stepsize in range(num_control_points-1, 0, -1):
        for start in range(0, num_control_points-stepsize):
            end = start + stepsize
            signature.append(distance(control_points[start], control_points[end]))
    # Normalize distances to compare trajectory shapes
    if normalize_distance:
        largest_distance = max(signature)
        signature = [0 if not largest_distance else d/largest_distance for d in signature]
    # Convert distances to a feature vector
    return convert_to_feature_vector(signature)

Now we are going to gather our points from file and assemble them into trajectories. We will use the same point reader and trajectory builder we used in previous examples. Then, we will computer the cluster labels.

In [3]:
from tracktable.domain.terrestrial import TrajectoryPointReader
from tracktable.source.trajectory import AssembleTrajectoryFromPoints
from tracktable.analysis.dbscan import compute_cluster_labels
from tracktable.core import data_directory
import os.path

data_filename = os.path.join(data_directory(), 'april_04_2013.csv')
inFile = open(data_filename, 'r')
reader = TrajectoryPointReader()
reader.input = inFile
reader.comment_character = '#'
reader.field_delimiter = ','
reader.object_id_column = 0
reader.timestamp_column = 1
reader.coordinates[0] = 2
reader.coordinates[1] = 3

builder = AssembleTrajectoryFromPoints()
builder.input = reader
builder.minimum_length = 5
builder.minimum_distance = 100
builder.minimum_time = 20

all_trajectories = list(builder)
# Get feature vectors for each trajectory describing their distance geometry
num_control_points = 4
feature_vectors = [distance_geometry_signature(trajectory, num_control_points, True)
                   for trajectory in all_trajectories]

# DBSCAN needs two parameters
#  1. Size of the box that defines when two points are close enough to one another to
#     belong to the same cluster.
#  2. Minimum number of points in a cluster
#
signature_length = len(feature_vectors[0])

# This is the default search box size. Feel free to change to fit your data.
search_box_span = [0.01] * signature_length
minimum_cluster_size = 5

cluster_labels = compute_cluster_labels(feature_vectors, search_box_span, minimum_cluster_size)

INFO: Trajectory assembly: New trajectories will be declared after a separation of None units or 0:30:00 seconds.
INFO: Trajectories with fewer than 5 points will be rejected.
(1) STATUS: 100 trajectories announced and 1 discarded for having fewer than 5 points
(1) STATUS: 200 trajectories announced and 3 discarded for having fewer than 5 points
(1) STATUS: 300 trajectories announced and 4 discarded for having fewer than 5 points
(1) STATUS: 400 trajectories announced and 5 discarded for having fewer than 5 points
(1) STATUS: 500 trajectories announced and 9 discarded for having fewer than 5 points
(1) STATUS: 600 trajectories announced and 10 discarded for having fewer than 5 points
(1) STATUS: 700 trajectories announced and 10 discarded for having fewer than 5 points
(1) STATUS: 800 trajectories announced and 11 discarded for having fewer than 5 points
(1) STATUS: 900 trajectories announced and 12 discarded for having fewer than 5 points
(1) STATUS: 1000 trajectories announced and 13

(1) STATUS: 9200 trajectories announced and 201 discarded for having fewer than 5 points
(1) STATUS: 9300 trajectories announced and 201 discarded for having fewer than 5 points
(1) STATUS: 9400 trajectories announced and 202 discarded for having fewer than 5 points
(1) STATUS: 9500 trajectories announced and 204 discarded for having fewer than 5 points
(1) STATUS: 9600 trajectories announced and 204 discarded for having fewer than 5 points
(1) STATUS: 9700 trajectories announced and 205 discarded for having fewer than 5 points
(1) STATUS: 9800 trajectories announced and 206 discarded for having fewer than 5 points
(1) STATUS: 9900 trajectories announced and 209 discarded for having fewer than 5 points
(1) STATUS: 10000 trajectories announced and 210 discarded for having fewer than 5 points
(3) STATUS: 10100 trajectories announced and 212 discarded for having fewer than 5 points
(3) STATUS: 10200 trajectories announced and 212 discarded for having fewer than 5 points
(3) STATUS: 10300 

(3) STATUS: 19100 trajectories announced and 415 discarded for having fewer than 5 points
(3) STATUS: 19200 trajectories announced and 416 discarded for having fewer than 5 points
(3) STATUS: 19300 trajectories announced and 416 discarded for having fewer than 5 points
(3) STATUS: 19400 trajectories announced and 417 discarded for having fewer than 5 points
(3) STATUS: 19500 trajectories announced and 417 discarded for having fewer than 5 points
(3) STATUS: 19600 trajectories announced and 417 discarded for having fewer than 5 points
(3) STATUS: 19700 trajectories announced and 417 discarded for having fewer than 5 points
(3) STATUS: 19800 trajectories announced and 418 discarded for having fewer than 5 points
(3) STATUS: 19900 trajectories announced and 418 discarded for having fewer than 5 points
(3) STATUS: 20000 trajectories announced and 419 discarded for having fewer than 5 points
(3) STATUS: 20100 trajectories announced and 420 discarded for having fewer than 5 points
(3) STATUS

(1) STATUS: 29000 trajectories announced and 980 discarded for having fewer than 5 points
(3) STATUS: 29100 trajectories announced and 983 discarded for having fewer than 5 points
(3) STATUS: 29200 trajectories announced and 988 discarded for having fewer than 5 points
(3) STATUS: 29300 trajectories announced and 996 discarded for having fewer than 5 points
(3) STATUS: 29400 trajectories announced and 998 discarded for having fewer than 5 points
(4) STATUS: 29470 trajectories announced and 1000 discarded for having fewer than 5 points
(3) STATUS: 29500 trajectories announced and 1000 discarded for having fewer than 5 points
(3) STATUS: 29600 trajectories announced and 1000 discarded for having fewer than 5 points
(3) STATUS: 29700 trajectories announced and 1002 discarded for having fewer than 5 points
(3) STATUS: 29800 trajectories announced and 1004 discarded for having fewer than 5 points
(3) STATUS: 29900 trajectories announced and 1006 discarded for having fewer than 5 points
(3) 

(1) STATUS: 38600 trajectories announced and 1875 discarded for having fewer than 5 points
(1) STATUS: 38700 trajectories announced and 1891 discarded for having fewer than 5 points
(2) STATUS: 38731 trajectories announced and 1900 discarded for having fewer than 5 points
(1) STATUS: 38800 trajectories announced and 1905 discarded for having fewer than 5 points
(1) STATUS: 38900 trajectories announced and 1920 discarded for having fewer than 5 points
(1) STATUS: 39000 trajectories announced and 1934 discarded for having fewer than 5 points
(3) STATUS: 39100 trajectories announced and 1937 discarded for having fewer than 5 points
(3) STATUS: 39200 trajectories announced and 1937 discarded for having fewer than 5 points
(3) STATUS: 39300 trajectories announced and 1938 discarded for having fewer than 5 points
(3) STATUS: 39400 trajectories announced and 1938 discarded for having fewer than 5 points
(3) STATUS: 39500 trajectories announced and 1938 discarded for having fewer than 5 points

(1) STATUS: 47300 trajectories announced and 2323 discarded for having fewer than 5 points
(1) STATUS: 47400 trajectories announced and 2332 discarded for having fewer than 5 points
(1) STATUS: 47500 trajectories announced and 2343 discarded for having fewer than 5 points
(1) STATUS: 47600 trajectories announced and 2354 discarded for having fewer than 5 points
(1) STATUS: 47700 trajectories announced and 2367 discarded for having fewer than 5 points
(1) STATUS: 47800 trajectories announced and 2380 discarded for having fewer than 5 points
(1) STATUS: 47900 trajectories announced and 2381 discarded for having fewer than 5 points
(1) STATUS: 48000 trajectories announced and 2394 discarded for having fewer than 5 points
(3) STATUS: 48100 trajectories announced and 2394 discarded for having fewer than 5 points
(3) STATUS: 48200 trajectories announced and 2396 discarded for having fewer than 5 points
(3) STATUS: 48300 trajectories announced and 2397 discarded for having fewer than 5 points

(1) STATUS: 56400 trajectories announced and 2753 discarded for having fewer than 5 points
(1) STATUS: 56500 trajectories announced and 2770 discarded for having fewer than 5 points
(1) STATUS: 56600 trajectories announced and 2786 discarded for having fewer than 5 points
(2) STATUS: 56699 trajectories announced and 2800 discarded for having fewer than 5 points
(1) STATUS: 56700 trajectories announced and 2800 discarded for having fewer than 5 points
(1) STATUS: 56800 trajectories announced and 2817 discarded for having fewer than 5 points
(1) STATUS: 56900 trajectories announced and 2829 discarded for having fewer than 5 points
(1) STATUS: 57000 trajectories announced and 2845 discarded for having fewer than 5 points
(3) STATUS: 57100 trajectories announced and 2847 discarded for having fewer than 5 points
(3) STATUS: 57200 trajectories announced and 2847 discarded for having fewer than 5 points
(3) STATUS: 57300 trajectories announced and 2849 discarded for having fewer than 5 points

(3) STATUS: 65300 trajectories announced and 3247 discarded for having fewer than 5 points
(3) STATUS: 65400 trajectories announced and 3249 discarded for having fewer than 5 points
(3) STATUS: 65500 trajectories announced and 3250 discarded for having fewer than 5 points
(3) STATUS: 65600 trajectories announced and 3250 discarded for having fewer than 5 points
(3) STATUS: 65700 trajectories announced and 3251 discarded for having fewer than 5 points
(3) STATUS: 65800 trajectories announced and 3252 discarded for having fewer than 5 points
(3) STATUS: 65900 trajectories announced and 3253 discarded for having fewer than 5 points
(3) STATUS: 66000 trajectories announced and 3254 discarded for having fewer than 5 points
(3) STATUS: 66100 trajectories announced and 3254 discarded for having fewer than 5 points
(3) STATUS: 66200 trajectories announced and 3254 discarded for having fewer than 5 points
(3) STATUS: 66300 trajectories announced and 3254 discarded for having fewer than 5 points

(3) STATUS: 74300 trajectories announced and 3535 discarded for having fewer than 5 points
(3) STATUS: 74400 trajectories announced and 3535 discarded for having fewer than 5 points
(3) STATUS: 74500 trajectories announced and 3536 discarded for having fewer than 5 points
(3) STATUS: 74600 trajectories announced and 3537 discarded for having fewer than 5 points
(3) STATUS: 74700 trajectories announced and 3537 discarded for having fewer than 5 points
(3) STATUS: 74800 trajectories announced and 3538 discarded for having fewer than 5 points
(3) STATUS: 74900 trajectories announced and 3539 discarded for having fewer than 5 points
(3) STATUS: 75000 trajectories announced and 3539 discarded for having fewer than 5 points
(3) STATUS: 75100 trajectories announced and 3539 discarded for having fewer than 5 points
(3) STATUS: 75200 trajectories announced and 3540 discarded for having fewer than 5 points
(3) STATUS: 75300 trajectories announced and 3540 discarded for having fewer than 5 points

Cluster Statistics    
Here we calculate the size of the clusters we labeled.

In [ ]:
# Assemble each cluster as a list of its component trajectories.
clusters = {}
for(vertex_id, cluster_id) in cluster_labels:
    if cluster_id not in clusters:
        clusters[cluster_id] = [all_trajectories[vertex_id]]
    else:
        clusters[cluster_id].append(all_trajectories[vertex_id])

# If a cluster does not have an id, it is an outlier
def cluster_name(cid):
    if cid == 0:
        return 'Outliers'
    else:
        return 'Cluster {}'.format(cid)

#Print the cluster id and the number of trajectories in the cluster.
print("RESULT: Cluster sizes:")
for(cid, cluster) in clusters.items():
    print("{}: {}".format(cluster_name(cid), len(cluster)))


Cluster Visualization    
You can use pyplot to see your clusters that were created

In [ ]:
from tracktable.examples.example_trajectory_rendering import render_trajectories
from tracktable.domain import terrestrial
from tracktable.render import mapmaker
from matplotlib import pyplot

sorted_ids = sorted(clusters.keys())
for cluster_id in sorted_ids:
    # Set up the canvas and map projection
    figure = pyplot.figure(figsize=[20, 15])
    axes = figure.add_subplot(1, 1, 1)
    (mymap, map_actors) = mapmaker.mapmaker(domain = 'terrestrial', map_name='region:conus')
    render_trajectories(mymap, clusters[cluster_id], trajectory_linewidth=1)
    figure.suptitle('{}: {} members'.format(cluster_name(cluster_id),
                                           len(clusters[cluster_id])))
    pyplot.show()